<a href="https://colab.research.google.com/github/2025-02-FML-team/WV-Team/blob/main/notebooks/03_model_build.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model Training

일단 피드백 대로 싱글몰트 카테고리만 추가적으로 더 도수에 따라서 카테고리를 나누었으며, 그것에 따라서 성능 측정을 할 계획입니다.
Colab을 이용 중이라고 하시면 content안에 미리 빌드된 파일을 넣거나, drive에 연결할때 까지 기다리면 됩니다.

### Method 1

In [11]:
import os
from pathlib import Path

try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    DATA_DIR = Path('/content/data')
else:
    DATA_DIR= Path(os.path.join(os.getcwd(), "../data/")).resolve()

In [12]:
import re
import glob
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# CSV 로드 및 정리, 본인 경로에 맞게 변환
CSV_PATH = DATA_DIR / 'whiskies_recategorized.csv'
IMAGE_SIZE = (256, 256)
RANDOM_STATE = 42

In [13]:
df = pd.read_csv(CSV_PATH, dtype={"id": str})
df["id"] = df["id"].astype(str).str.strip().str.replace(r"\.0$", "", regex=True)
df["category"] = df["category"].astype(str).str.strip()
paths = [DATA_DIR / p for p in df["local_full_path"]]

# 이미지 로드
X_list = []
for p in paths:
    with Image.open(p) as im:
        im = im.convert("RGB")
        im = im.resize(IMAGE_SIZE)
        arr = np.asarray(im, dtype=np.uint8)
        X_list.append(arr)
X = np.stack(X_list, axis=0)


# 라벨 인코딩
labels = df["category"].values
le = LabelEncoder()
y_int = le.fit_transform(labels)

# train/test 분리
train_images, test_images, train_labels, test_labels = train_test_split(
    X, y_int, test_size=0.2, random_state=RANDOM_STATE, stratify=y_int
)

print("train_images:", train_images.shape)
print("test_images:", test_images.shape)
print("train_labels 분포:", np.bincount(train_labels))
print("class mapping:", dict(zip(le.classes_, range(len(le.classes_)))))

train_images: (2354, 256, 256, 3)
test_images: (589, 256, 256, 3)
train_labels 분포: [395 185 121 585  61 184 194 629]
class mapping: {'Blended': 0, 'Bourbon': 1, 'Brandy': 2, 'Other': 3, 'Rye': 4, 'SM_40_43': 5, 'SM_43_46': 6, 'SM_G46': 7}


여기서부터는 강의록2에 있는 코드입니다.

위에 있는 3가지 방법중에 하나를 고르고 아래 코드를 실행시키면 됩니다.

In [14]:
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential([
    keras.Input(shape=(256, 256, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(8, activation='softmax'),
])

In [15]:
model.compile(optimizer= "rmsprop",
              loss = "sparse_categorical_crossentropy",
              metrics=["accuracy"])

방법2를 골랐을 경우 128 * 128 * 3을 128 * 256 * 3으로 바꿔주세요

In [16]:
# train_images = train_images.reshape((train_images.shape[0], 256*256*3))
# test_images  = test_images.reshape((test_images.shape[0], 256*256*3))
train_images = train_images.astype("float32")/255
test_images = test_images.astype("float32")/255

In [17]:
print(train_images.shape, train_images.dtype)
print(train_labels.shape, train_labels[:10])

(2354, 256, 256, 3) float32
(2354,) [7 3 4 7 6 2 3 4 3 3]


In [18]:
model.fit(train_images, train_labels, epochs=50, batch_size=128)

Epoch 1/50


2025-11-12 17:14:25.613736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


19/19 ━━━━━━━━━━━━━━━━━━━━ 46s 2s/step - accuracy: 0.2523 - loss: 17.4192
Epoch 2/50
 2/19 ━━━━━━━━━━━━━━━━━━━━ 44s 3s/step - accuracy: 0.1895 - loss: 1.9569


KeyboardInterrupt



In [32]:
test_loss, test_acc = model.evaluate(test_images, test_labels)
print(test_acc)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step - accuracy: 0.2732 - loss: 7.6455
0.301075279712677
